In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import os
import matplotlib.font_manager as fm
import yfinance as yf
import datetime
from datetime import timedelta, datetime
import openpyxl
import setuptools
import pandas_datareader as pdr

In [2]:
df_GARE13 = pd.read_excel('GARE13_22-08.xlsx')
GARE11_base = pd.read_excel('GARE11_18-08.xlsx')
gare11_11x8 = pd.read_excel('GARE11_11-08.xlsx')
gare11_12x8 = pd.read_excel('GARE11_12-08.xlsx')
gare11_14x8 = pd.read_excel('GARE11_14-08.xlsx')
df_GARE12 = pd.read_excel('GARE12_08.08.xlsx')

In [3]:
GARE11_base

,DOCUMENTO,NOME,PF_PJ,TOTAL_NOVO,FINANCEIRO_NOVO,CATEGORIAS ATUAIS
0,6556,GLEDSON ANDRADE SANTOS,PF,6200,56358.00,50k-100k
1,80527,HUGO LEONARDO LEMOS MATOS,PF,24,218.16,0-1k
2,114790,HANS HERBERT LAUBMEYER FILHO,PF,360,3272.40,1k-5k
3,200018,VERIDIANE CRISTINA ZACARIA,PF,2,18.18,0-1k
4,549258,NICOLE PEREIRA DA COSTA,PF,5,45.45,0-1k
...,...,...,...,...,...,...
394618,99346770678,CLAUDIANO TABORDA COELHO,PF,10,90.90,0-1k
394619,99357720120,SERGIO LUCENA RESENDE,PF,1,9.09,0-1k
394620,99452391200,THIAGO ROCHA DE MENEZES,PF,1,9.09,0-1k
394621,99830507068,RODRIGO PIRES ACOSTA,PF,109,990.81,0-1k


In [4]:
df_analise = df_GARE13
df_analise['COTISTA_GARE11'] = df_analise['DOCUMENTO'].apply(lambda doc: 'GARE11' if doc in GARE11_base['DOCUMENTO'].values else 'AUSENTE')
df_analise['COTISTA_GARE12'] = df_analise['DOCUMENTO'].apply(lambda doc: 'GARE12' if doc in df_GARE12['DOCUMENTO'].values else 'AUSENTE')
df_analise['PRESENTE_11x8'] = df_analise['DOCUMENTO'].apply(lambda doc: 'PRESENTE' if doc in gare11_11x8['DOCUMENTO'].values else 'AUSENTE')
df_analise['PRESENTE_12x8'] = df_analise['DOCUMENTO'].apply(lambda doc: 'PRESENTE' if doc in gare11_12x8['DOCUMENTO'].values else 'AUSENTE')
df_analise['PRESENTE_14x8'] = df_analise['DOCUMENTO'].apply(lambda doc: 'PRESENTE' if doc in gare11_14x8['DOCUMENTO'].values else 'AUSENTE')
df_analise_merge = pd.merge(df_GARE13, GARE11_base[['DOCUMENTO', 'CATEGORIAS ATUAIS']],
                            on = ['DOCUMENTO'],
                            how = 'left',
                            suffixes=('', '_GARE11'))
df_analise_merge['CATEGORIAS_ATUAIS'] = df_analise_merge['CATEGORIAS ATUAIS'].fillna(0)

In [5]:
gare13_cat = df_analise_merge.groupby(['CATEGORIAS_ATUAIS'])['TOTAL'].sum()
gare13_cat_sorted = gare13_cat.sort_values(ascending=False)
gare13_dataframe = pd.DataFrame(gare13_cat_sorted)
gare13_dataframe = gare13_dataframe.reset_index()
gare13_dataframe

,CATEGORIAS_ATUAIS,TOTAL
0,0,393335
1,>15M,353715
2,100k-500k,258041
3,20k-50k,186125
4,5k-20k,170531
5,50k-100k,159206
6,1k-5k,129805
7,0-1k,63825
8,500k-1M,32336


In [6]:
gare13_cat = df_analise_merge.groupby(['CATEGORIAS_ATUAIS'])['TOTAL'].count()
gare13_cat_sorted = gare13_cat.sort_values(ascending=False)
gare13_dataframe = pd.DataFrame(gare13_cat_sorted)
gare13_dataframe = gare13_dataframe.reset_index()
gare13_dataframe

,CATEGORIAS_ATUAIS,TOTAL
0,0-1k,5353
1,1k-5k,2067
2,5k-20k,730
3,0,339
4,20k-50k,268
5,50k-100k,84
6,100k-500k,53
7,500k-1M,4
8,>15M,1


In [ ]:
df_ausente = df_analise_merge.query('COTISTA_GARE11 == "AUSENTE"')
gare12_secundario = df_analise_merge.query('COTISTA_GARE12 == "AUSENTE"')

,CHAVE,DOCUMENTO,NOME,PF_PJ,TOTAL,COTISTA_GARE11,COTISTA_GARE12,PRESENTE_11x8,PRESENTE_12x8,PRESENTE_14x8,CATEGORIAS ATUAIS,CATEGORIAS_ATUAIS
2,00008799750ANDRE BAPTISTA BARCAUI,8799750,ANDRE BAPTISTA BARCAUI,PF,1,AUSENTE,AUSENTE,AUSENTE,AUSENTE,AUSENTE,NaN,0
30,00126679193STEPHANIE MIRANDA IMBROISI CAP,126679193,STEPHANIE MIRANDA IMBROISI CAPONE,PF,6000,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
53,00227230647GLAUCIANA FERNANDES PEIXOTO DE,227230647,GLAUCIANA FERNANDES PEIXOTO DE SIQUEIRA,PF,1688,AUSENTE,AUSENTE,PRESENTE,PRESENTE,PRESENTE,NaN,0
60,00255340680CARLA VILAS BOAS,255340680,CARLA VILAS BOAS,PF,5,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
104,00364109777ELIZABETH COHEN,364109777,ELIZABETH COHEN,PF,4354,AUSENTE,GARE12,PRESENTE,PRESENTE,AUSENTE,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8848,98104560778AUGUSTO CESAR TEIXEIRA LEITE,98104560778,AUGUSTO CESAR TEIXEIRA LEITE,PF,10,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
8861,98618652768CARLOS ALBERTO BOURGUIGNON,98618652768,CARLOS ALBERTO BOURGUIGNON,PF,1006,AUSENTE,AUSENTE,AUSENTE,AUSENTE,AUSENTE,NaN,0
8862,98680196053ROBSON DIAS FLORES,98680196053,ROBSON DIAS FLORES,PF,50,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
8870,98888242953GILSON SAKUMA DE OLIVEIRA,98888242953,GILSON SAKUMA DE OLIVEIRA,PF,10,AUSENTE,AUSENTE,AUSENTE,AUSENTE,AUSENTE,NaN,0


In [ ]:
df_analise_merge.groupby(['COTISTA_GARE12'])['TOTAL'].count()   

COTISTA_GARE12
AUSENTE     863
GARE12     8036
Name: TOTAL, dtype: int64

In [27]:
doc = 'DOCUMENTO'
nome = 'NOME'
tipo = 'PF_PJ'
qtde_cotas = 'TOTAL'
gare11 = 'GARE11'
gare12 = 'GARE12'
cat = 'CATEGORIAS_ATUAIS'

t10_GARE13 = df_analise_merge.sort_values(by=['TOTAL'], ascending=False).iloc[:10, :]

In [28]:
t10_GARE13 = t10_GARE13.drop(columns=['CHAVE', 'CATEGORIAS ATUAIS'])

In [29]:
t10_GARE13

,DOCUMENTO,NOME,PF_PJ,TOTAL,COTISTA_GARE11,COTISTA_GARE12,PRESENTE_11x8,PRESENTE_12x8,PRESENTE_14x8,CATEGORIAS_ATUAIS
8830,97521225000125,MAXI RENDA FUNDO DE INVESTIMENTO IMOBILIARIO -...,PJ,353715,GARE11,GARE12,PRESENTE,PRESENTE,PRESENTE,>15M
1676,4286442608,LEONARDO CARVALHO NASCIMENTO,PF,37675,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,0
3105,8473412168,ROBERTO ALMEIDA SILVA,PF,20816,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,0
8650,91770815791,ANA LUCIA GONCALVES DURAO,PF,19594,AUSENTE,GARE12,PRESENTE,PRESENTE,PRESENTE,0
7081,44000693700,LUIZ CARLOS NIGRI,PF,19529,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,0
4464,14690276072,PLINIO LUIZ BORTONCELLO,PF,18742,GARE11,GARE12,PRESENTE,PRESENTE,PRESENTE,500k-1M
1206,3017094910,LUANA VON LINSINGEN PASETCHNY,PF,16000,GARE11,GARE12,PRESENTE,PRESENTE,PRESENTE,100k-500k
7917,66710197700,JOSE DE ANDRADE FORTES,PF,15317,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,0
7599,55020950700,MARIA VITORIA DE ANDRADE FORTES,PF,14907,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,0
8289,80298230704,CELIA MARIA CORREA FORTES,PF,14475,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,0


In [8]:
gare11_gare12_11x8 = df_analise_merge.query("COTISTA_GARE11 == 'GARE11' and COTISTA_GARE12 == 'GARE12' and PRESENTE_11x8 == 'PRESENTE'")
gare12_11x8 = df_analise_merge.query("COTISTA_GARE11 == 'AUSENTE' and COTISTA_GARE12 == 'GARE12' and PRESENTE_11x8 == 'PRESENTE'")
gare11_11x8_12x8 = df_analise_merge.query("PRESENTE_11x8 == 'PRESENTE' and COTISTA_GARE12 == 'GARE12' and PRESENTE_14x8 == 'AUSENTE'")
#gare11_gare12_11x8 = df_analise_merge.query("COTISTA_GARE11 == 'GARE11' and COTISTA_GARE12 == 'GARE12' and PRESENTE_11x8 == 'PRESENTE'")

In [14]:
x = gare11_gare12_11x8['TOTAL'].count()
y = gare12_11x8['TOTAL'].count()
z = gare11_11x8_12x8['TOTAL'].count()
print(x)
print(y)
print(z)

7863
68
52


In [18]:
t10_NC = df_ausente.sort_values(by=['TOTAL'], ascending=False).iloc[:10, :]
t10_NC

,CHAVE,DOCUMENTO,NOME,PF_PJ,TOTAL,COTISTA_GARE11,COTISTA_GARE12,PRESENTE_11x8,PRESENTE_12x8,PRESENTE_14x8,CATEGORIAS ATUAIS,CATEGORIAS_ATUAIS
1676,04286442608LEONARDO CARVALHO NASCIMENTO,4286442608,LEONARDO CARVALHO NASCIMENTO,PF,37675,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
3105,08473412168ROBERTO ALMEIDA SILVA,8473412168,ROBERTO ALMEIDA SILVA,PF,20816,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
8650,91770815791ANA LUCIA GONCALVES DURAO,91770815791,ANA LUCIA GONCALVES DURAO,PF,19594,AUSENTE,GARE12,PRESENTE,PRESENTE,PRESENTE,NaN,0
7081,44000693700LUIZ CARLOS NIGRI,44000693700,LUIZ CARLOS NIGRI,PF,19529,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
7917,66710197700JOSE DE ANDRADE FORTES,66710197700,JOSE DE ANDRADE FORTES,PF,15317,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
7599,55020950700MARIA VITORIA DE ANDRADE FORTE,55020950700,MARIA VITORIA DE ANDRADE FORTES,PF,14907,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
8289,80298230704CELIA MARIA CORREA FORTES,80298230704,CELIA MARIA CORREA FORTES,PF,14475,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
4847,18357377734MARIA BEATRIZ DE ANDRADE FORTE,18357377734,MARIA BEATRIZ DE ANDRADE FORTES,PF,13643,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0
831,02102603784VANESSA S VARAO MONTEIRO,2102603784,VANESSA S VARAO MONTEIRO,PF,13564,AUSENTE,GARE12,PRESENTE,PRESENTE,AUSENTE,NaN,0
980,02461668787MARCIO JOAO DE ANDRADE FORTES,2461668787,MARCIO JOAO DE ANDRADE FORTES,PF,10604,AUSENTE,GARE12,AUSENTE,AUSENTE,AUSENTE,NaN,0


In [17]:
pasta_destino = r'G:\.shortcut-targets-by-id\1YZLLTFOuLniE6hO7WNHXl2c3MSa4TOv6\X. GAME11\GUARDIAN Institucional\5. Planejamento & Processos\2. 2023\4. Research\2025\5. Inteligência Passivos Guardian\1. ARQUIVOS B3\BRUTOS\GARE13'

os.makedirs(pasta_destino, exist_ok=True)
caminho_arq_excel = os.path.join(pasta_destino, 'Gare13_Completo.xlsx')
df_analise_merge.to_excel(caminho_arq_excel, index=False)

print('Arquivo salvo com sucesso')

Arquivo salvo com sucesso
